###  Multiple financial services API calls with data saved to Google Sheets.  
This notebooks calls the Polygon API and gets basic stock information, one call for each stock you own.  It then saves this data to a Google Sheet.  Upload the notebook to https://www.nbfire.io/ and run it every night on a schedule to automate data collection.

You will need a basic subscription at https://polygon.io with an API key.  Enter the key as a parameter at runtime
To set up google sheets please follow https://ploomber.io/blog/gsheets/.  Again, enter the Google Cloud credentials at runtime.  

To get going you will need a google sheet linked to your google cloud account.  You can find instructions here https://www.nbfire.io/resources/googlesheets.  In your Google Sheet you will also need to create a worksheet called "Investments" with with a column headed 'Abbr'.  Add the NYSE abbreviation for the stock you wish to track into this column.  So for Apple it is APPL.  You can find a full list here https://eoddata.com/symbols.aspx

You can find a full explaination and how to guide here

In [32]:
# To be on the safe side we have included pip installation of the packages needed to run this notebook because https://www.nbfire.io/ is a 
# containerised environment at might not be able to install them
!pip install requests
!pip install gspread
!pip install pandas
!pip install datetime

In [33]:
import requests
import json
import gspread
import pandas
from datetime import datetime
import pandas as pd
import time

In [34]:
# To make it easier to update and share this notebook we will be entering sensitive keys are runtime using nbfire.  T
# This cell is a placeholder for these values.  This is why the cell has a 'parameters' tag added.  For more
#information please read https://papermill.readthedocs.io/en/latest/usage-parameterize.html
credentials = ""
API_Key = ""

In [35]:
# If you are working locally comment out this cell and insert your credentials and API key in the cell above.  Once
# you want to move the notebook to NBFire uncomment this cell and replace "credentials" with "creds" in the cell 
# below

creds = json.loads(credentials)

In [36]:
#let's connect to google sheets using our credentials (or creds, see above!)
sa = gspread.service_account_from_dict(creds)

In [37]:
#Get the current date and time in an easy to read format which we can add to the data we 
#are fetching and storing in our worksheets
current_datetime = datetime.now()
now = current_datetime.strftime("%m/%d/%Y, %H:%M:%S")



In [39]:
#This function gets a list of the worksheet titles from our Google Sheet.  Each worksheet corresponds to 
# a stock we hold
def get_worksheets():
    worksheet_objs = sheet.worksheets()
    worksheets_list = []
    for worksheet in worksheet_objs:
        worksheets_list.append(worksheet.title)
    return(worksheets_list)



In [40]:
#This function checks if you have a corresponding worksheet for the stocks you have listed in the investments
#worksheet and and if not creates it
def add_stock(my_stocks):
    for s in my_stocks:
        if s not in all_investments:
            add_worksheet(s)

In [41]:
#This function adds a new worksheet with the stock abbreviation as its name
def add_worksheet(s):
  sheet.add_worksheet(title=s, rows="100", cols="20")
  create_worksheet_headings(s) 


In [42]:
#This function creates all the column headings we need in a new worksheet
def create_worksheet_headings(s):
  worksheet = sheet.worksheet(s)
  worksheet.update_cell(1, 1, 'Timestamp')
  worksheet.update_cell(1, 2, 'Opening')
  worksheet.update_cell(1, 3, 'Closing')
  worksheet.update_cell(1, 4, 'Volume')
  worksheet.update_cell(1, 5, 'Value')

In [43]:
#This function takes a stock and returns the corresponding number of shares with have in our 'investments' worksheet
def get_shares(s):
    print(s)
    my_stock = investments.find(s)
    my_shares = investments.cell(my_stock.row, my_stock.col + 1).value
    return(my_shares)


In [44]:
#This function takes a list of your stocks, makes a call to the Polygon API for each and  gets the opening 
#and closing prices from the previous day plus the volume of trades.  I have added a sleep because we are using the
#FREE, rate limited, Polygon account. If you want to use this for real you might want to consider using a paid 
#version.  If you hit the limit you will get a 'key error'

def get_stocks(stocks):
  for stock in stocks:
    url = "https://api.polygon.io/v2/aggs/ticker/" + stock + "/prev?adjusted=true&apiKey=" + API_Key
    x = requests.get(url) 
    data = x.json()
    results = data['results'][0]   
    opening = results['o']
    closing = results['c']
    volume = results['v']
    shares = get_shares(stock)
    value = (float(shares) * closing)
    new_entry =[now, opening, closing, volume, value]
    print(new_entry)
    update_worksheet(stock, new_entry)
    time.sleep(10)
    

In [45]:
# This helper function updates a worksheet with new data
def update_worksheet(s, data):
    worksheet = sheet.worksheet(s)
    worksheet.append_row(data)

In [46]:
#Let's open our Google Sheet and get our list of investments.  I am using the Google sheet's key from sheet's url
sheet = sa.open_by_key('1P1b5MjvJ3fvbdqLd8lXQhtRRRZK8G55UJvbX9sPKsUU')
investments = sheet.worksheet("Investments")


In [38]:
#We can use Pandas to get a list of the stocks we hold from our investment worksheet and then
#get a list of all our worksheets.

df = pd.DataFrame(investments.get_all_records())
my_stocks = (df['Abbr'].to_list())
all_investments = get_worksheets()

In [47]:
#Let's run everything.  We use "add_stock" to check whether any new rows have been added to our 
#Investments (and create a corresponding worksheet if neccessary).  We then run "get_stocks" to get the latest 
#values from Polygon, calculate the value of our investments and update all the worksheets
add_stock(my_stocks)
get_stocks(my_stocks)



AB
['03/19/2024, 15:25:22', 33.37, 33.48, 183248, 3347.9999999999995]
NAT
['03/19/2024, 15:25:22', 4.06, 4.03, 4351568.0, 403.0]
NKE
['03/19/2024, 15:25:22', 98.98, 98.74, 7206466.0, 6911.799999999999]
XOM
['03/19/2024, 15:25:22', 111.79, 112.3, 16770146.0, 8984.0]
BBW
['03/19/2024, 15:25:22', 29.84, 28.93, 346987, 2603.7]
AAPL
['03/19/2024, 15:25:22', 175.57, 173.72, 75606556.0, 7817.4]
AAT
['03/19/2024, 15:25:22', 21.17, 21.11, 338221, 211.1]
AC
['03/19/2024, 15:25:22', 33.35, 33.33, 3852, 66660.0]
ACI
['03/19/2024, 15:25:22', 20.73, 20.82, 1867398.0, 10410.0]
